# Tokenizers库的使用介绍

tokenizers库是HuggingFace使用Rust的实现的一个同时针对研究与生产的文本tokenization的库。同时它也是transformers库中FastTokenizer背后的实现。

# 从零开始构建一个Tokenizer

## 构建Tokenizer类与Trainer

In [1]:
from tokenizers import Tokenizer
from tokenizers.models import BPE

tokenizer = Tokenizer(BPE(unk_token="[UNK]"))

In [2]:
# BpeTrainer, UnigramTrainer, WordLevelTrainer, WordPieceTrainer
from tokenizers.trainers import BpeTrainer

# 写入特殊标记列表的顺序很重要：在这里， "[UNK]" 将获得 ID 0， "[CLS]" 将获得 ID 1，以此类推。
trainer = BpeTrainer(special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"])

## 添加Pre-tokenizer

In [3]:
# 使用PreTokenizer可以确保我们最终不会生成 word与word级别的合并
from tokenizers.pre_tokenizers import Whitespace

tokenizer.pre_tokenizer = Whitespace()

## 训练

首先我们需要先下载 [wikitext-103](https://blog.einstein.ai/the-wikitext-long-term-dependency-language-modeling-dataset/) 数据集，并且解压到`data`目录下。

```bash
mkdir data && cd data
wget https://s3.amazonaws.com/research.metamind.io/wikitext/wikitext-103-raw-v1.zip
unzip wikitext-103-raw-v1.zip
```

In [4]:
files = [
    f"./data/wikitext-103-raw/wiki.{split}.raw" for split in ["test", "train", "valid"]
]
tokenizer.train(files, trainer)

## 保存训练好的Tokenizer

In [5]:
tokenizer.save("./data/wikitext-103-raw/tokenizer-wiki.json")

## 从文件加载Tokenizer

In [6]:
tokenizer = Tokenizer.from_file("./data/wikitext-103-raw/tokenizer-wiki.json")

## 使用Tokenizer

In [7]:
output = tokenizer.encode("Hello, y'all! How are you 😁 ?")

In [8]:
output

Encoding(num_tokens=11, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])

In [9]:
print(output.tokens)

['Hello', ',', 'y', "'", 'all', '!', 'How', 'are', 'you', '[UNK]', '?']


In [10]:
print(output.ids)

[27253, 16, 93, 11, 5097, 5, 7961, 5112, 6218, 0, 35]


In [11]:
print(output.offsets)

[(0, 5), (5, 6), (7, 8), (8, 9), (9, 12), (12, 13), (14, 17), (18, 21), (22, 25), (26, 27), (28, 29)]


## 添加后处理

我们指定句子对模板，其形式应为 "[CLS] $A [SEP] $B [SEP]" ，其中 $A 代表第一句， $B 代表第二句。模板中添加的 :1 代表我们希望输入的每一部分的 type IDs ：默认情况下，所有内容的 都为 0（这也是我们没有 $A:0 的原因），这里我们将第二句的标记和最后一个 "[SEP]" 标记设置为 1。

In [12]:
from tokenizers.processors import TemplateProcessing

tokenizer.post_processor = TemplateProcessing(
    single="[CLS] $A [SEP]",
    pair="[CLS] $A [SEP] $B:1 [SEP]:1",
    special_tokens=[
        ("[CLS]", tokenizer.token_to_id("[CLS]")),
        ("[SEP]", tokenizer.token_to_id("[SEP]")),
    ],
)

## 再来检查编码结果

In [13]:
output = tokenizer.encode("Hello, y'all! How are you 😁 ?, ab")
print(output.tokens)

['[CLS]', 'Hello', ',', 'y', "'", 'all', '!', 'How', 'are', 'you', '[UNK]', '?', ',', 'ab', '[SEP]']


In [14]:
output = tokenizer.encode("this is sentence 1", "this is sentence 2")
print(output.tokens)
print(output.type_ids)

['[CLS]', 'this', 'is', 'sentence', '1', '[SEP]', 'this', 'is', 'sentence', '2', '[SEP]']
[0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1]


## 批量Encode - `encode_batch`

In [15]:
output = tokenizer.encode_batch(
    [
        ["Hello, y'all!", "How are you 😁 ?"],
        [
            "Hello to you too!",
            "I'm fine, thank you!",
        ],
    ]
)
output

[Encoding(num_tokens=14, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing]),
 Encoding(num_tokens=16, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])]

In [16]:
tokenizer.enable_padding(pad_id=3, pad_token="[PAD]")
output = tokenizer.encode_batch(["Hello, y'all!", "How are you 😁 ?"])
print(output[1].tokens)
print(output[1].attention_mask)

['[CLS]', 'How', 'are', 'you', '[UNK]', '?', '[SEP]', '[PAD]']
[1, 1, 1, 1, 1, 1, 1, 0]


# Normalizer

In [17]:
from tokenizers import normalizers
from tokenizers.normalizers import NFD, StripAccents

normalizer = normalizers.Sequence([NFD(), StripAccents()])

In [18]:
normalizer.normalize_str("Héllò hôw are ü?")

'Hello how are u?'

# Pre-Tokenization

In [19]:
from tokenizers import pre_tokenizers
from tokenizers.pre_tokenizers import Digits, Whitespace

pre_tokenizer = pre_tokenizers.Sequence([Whitespace(), Digits(individual_digits=True)])

In [20]:
pre_tokenizer.pre_tokenize_str("Call 911")

[('Call', (0, 4)), ('9', (5, 6)), ('1', (6, 7)), ('1', (7, 8))]

# BertTokenizer from Scatch

In [21]:
from tokenizers import Tokenizer, normalizers
from tokenizers.normalizers import NFD, Lowercase, StripAccents
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.models import WordPiece
from tokenizers.processors import TemplateProcessing

In [22]:
bert_uncased_tokenizer = Tokenizer(WordPiece(unk_token="[UNK]"))
bert_uncased_tokenizer.normalizer = normalizers.Sequence(
    [NFD(), Lowercase(), StripAccents()]
)
bert_uncased_tokenizer.pre_tokenizer = Whitespace()
bert_uncased_tokenizer.post_processor = TemplateProcessing(
    single="[CLS] $A [SEP]",
    pair="[CLS] $A [SEP] $B:1 [SEP]:1",
    special_tokens=[
        ("[CLS]", 1),
        ("[SEP]", 2),
    ],
)

In [23]:
from tokenizers.trainers import WordPieceTrainer

trainer = WordPieceTrainer(
    vocab_size=30522, special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"]
)

files = [
    f"./data/wikitext-103-raw/wiki.{split}.raw" for split in ["test", "train", "valid"]
]
bert_uncased_tokenizer.train(files, trainer)
bert_uncased_tokenizer.save("./data/wikitext-103-raw/bert-wiki.json")

In [24]:
output = bert_uncased_tokenizer.encode("Welcome to the 🤗 Tokenizers library.")
print(output.tokens)
print(output.ids)

['[CLS]', 'welcome', 'to', 'the', '[UNK]', 'tok', '##eni', '##zer', '##s', 'library', '.', '[SEP]']
[1, 18263, 7128, 7108, 0, 22453, 27107, 12800, 4073, 11046, 18, 2]


In [25]:
bert_uncased_tokenizer.decode(output.ids)

'welcome to the tok ##eni ##zer ##s library .'

In [26]:
from tokenizers import decoders

bert_uncased_tokenizer.decoder = decoders.WordPiece()
bert_uncased_tokenizer.decode(output.ids)

'welcome to the tokenizers library.'

# 从一个数据迭代器来训练

In [27]:
from tokenizers import (
    Tokenizer,
    decoders,
    models,
    normalizers,
    pre_tokenizers,
    trainers,
)

tokenizer = Tokenizer(models.Unigram())
tokenizer.normalizer = normalizers.NFKC()
tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel()
tokenizer.decoder = decoders.ByteLevel()
trainer = trainers.UnigramTrainer(
    vocab_size=20000,
    initial_alphabet=pre_tokenizers.ByteLevel.alphabet(),
    special_tokens=["<PAD>", "<BOS>", "<EOS>"],
)

In [28]:
# First few lines of the "Zen of Python" https://www.python.org/dev/peps/pep-0020/
data = [
    "Beautiful is better than ugly."
    "Explicit is better than implicit."
    "Simple is better than complex."
    "Complex is better than complicated."
    "Flat is better than nested."
    "Sparse is better than dense."
    "Readability counts."
]
tokenizer.train_from_iterator(data, trainer=trainer)

In [29]:
import datasets

datasets.logging.set_verbosity_error()

dataset = datasets.load_dataset(
    "wikitext", "wikitext-103-raw-v1", split="train+test+validation"
)


def batch_iterator(batch_size=1000):
    for i in range(0, len(dataset), batch_size):
        yield dataset[i : i + batch_size]["text"]


tokenizer.train_from_iterator(batch_iterator(), trainer=trainer, length=len(dataset))

# 在一个旧的Tokenizer上训练

我们可以直接使用`transfomers`库中的`AutoTokenizer.train_new_from_iterator()`来在一个已有的Tokenizer上用新数据进行训练。新的Tokenizer除了内部的分词器模型的相关词表等不同外，其余均与原分词器一致。

下面我们将演示在`gpt-2`的分词器上，为Python代码来训练一个专门的分词器。CodeSearchNet 数据集是为 CodeSearchNet 挑战赛创建的，包含来自 GitHub 上多个编程语言开源库的数百万个函数。在此，我们将加载该数据集的 Python 部分：

In [30]:
from datasets import load_dataset

raw_dataset = load_dataset("code_search_net", "python", split="train")

我们可以看到数据集中包括了许多的字段，我们这里面使用`whole_func_string`字段，它包括了函数的代码以及函数的注释。

In [31]:
raw_dataset

Dataset({
    features: ['repository_name', 'func_path_in_repository', 'func_name', 'whole_func_string', 'language', 'func_code_string', 'func_code_tokens', 'func_documentation_string', 'func_documentation_tokens', 'split_name', 'func_code_url'],
    num_rows: 412178
})

In [32]:
def get_training_corpus():
    return (
        raw_dataset[i : i + 1024]["whole_func_string"]
        for i in range(0, len(raw_dataset), 1024)
    )


training_corpus = get_training_corpus()

In [33]:
from transformers import AutoTokenizer

old_tokenizer = AutoTokenizer.from_pretrained("gpt2")

In [34]:
example = '''def add_numbers(a, b):
    """Add the two numbers `a` and `b`."""
    return a + b'''

tokens = old_tokenizer.tokenize(example)
print(tokens)

['def', 'Ġadd', '_', 'n', 'umbers', '(', 'a', ',', 'Ġb', '):', 'Ċ', 'Ġ', 'Ġ', 'Ġ', 'Ġ"""', 'Add', 'Ġthe', 'Ġtwo', 'Ġnumbers', 'Ġ`', 'a', '`', 'Ġand', 'Ġ`', 'b', '`', '."', '""', 'Ċ', 'Ġ', 'Ġ', 'Ġ', 'Ġreturn', 'Ġa', 'Ġ+', 'Ġb']


这个Tokenizer有一些特殊符号，如 Ġ 和 Ċ ，分别表示空格和换行。我们可以看到，这样做的效率并不高：Tokenizer会为每个空格返回单独的标记符号，而它本可以将缩进级别分组（因为在代码中，4 个或 8 个空格是很常见的）。由于不习惯看到带有 _ 字符的单词，它对函数名的分割也有点奇怪。

让我们来训练一个新的标记符，看看它是否能解决这些问题。

In [35]:
tokenizer = old_tokenizer.train_new_from_iterator(training_corpus, 52000)

In [36]:
tokens = tokenizer.tokenize(example)
print(tokens)

['def', 'Ġadd', '_', 'numbers', '(', 'a', ',', 'Ġb', '):', 'ĊĠĠĠ', 'Ġ"""', 'Add', 'Ġthe', 'Ġtwo', 'Ġnumbers', 'Ġ`', 'a', '`', 'Ġand', 'Ġ`', 'b', '`."""', 'ĊĠĠĠ', 'Ġreturn', 'Ġa', 'Ġ+', 'Ġb']


在这里，我们再次看到了表示空格和换行的特殊符号 Ġ 和 Ċ ，但我们也可以看到，我们的tokenizer学习了一些对 Python 函数语料库来说非常特殊的标记：例如，有一个 `ĊĠĠĠ` 标记表示缩进，还有一个 `Ġ"""` 标记表示文档字符串开始的三个引号。标记符还能正确分割 `_` 上的函数名。这是一个相当紧凑的表示法；

In [37]:
example = """class LinearLayer():
    def __init__(self, input_size, output_size):
        self.weight = torch.randn(input_size, output_size)
        self.bias = torch.zeros(output_size)

    def __call__(self, x):
        return x @ self.weights + self.bias
    """
print(tokenizer.tokenize(example))

['class', 'ĠLinear', 'Layer', '():', 'ĊĠĠĠ', 'Ġdef', 'Ġ__', 'init', '__(', 'self', ',', 'Ġinput', '_', 'size', ',', 'Ġoutput', '_', 'size', '):', 'ĊĠĠĠĠĠĠĠ', 'Ġself', '.', 'weight', 'Ġ=', 'Ġtorch', '.', 'randn', '(', 'input', '_', 'size', ',', 'Ġoutput', '_', 'size', ')', 'ĊĠĠĠĠĠĠĠ', 'Ġself', '.', 'bias', 'Ġ=', 'Ġtorch', '.', 'zeros', '(', 'output', '_', 'size', ')', 'ĊĊĠĠĠ', 'Ġdef', 'Ġ__', 'call', '__(', 'self', ',', 'Ġx', '):', 'ĊĠĠĠĠĠĠĠ', 'Ġreturn', 'Ġx', 'Ġ@', 'Ġself', '.', 'weights', 'Ġ+', 'Ġself', '.', 'bias', 'ĊĠĠĠĠ']


对于上面这个例子，我们可以看到双缩进的标记：`ĊĠĠĠĠĠĠĠ`，以及像`class`，`init`，`call`，`self`这样的特珠python关键字也会被标记为一个token，我们可以看到，除了对 _ 和 . 进行拆分外，标记符还能正确拆分驼峰字母的名称： LinearLayer 被标记为 ["ĠLinear", "Layer"] 。

# tokenizers库的主要组件

<div align="center">
  <img src="./assets/tokenizers.png" width="600"/> </div>
